In [0]:
# Parameters
path_book = "/Volumes/workspace/dataanalysispysparkbook/bronze_files/*.txt"
path_one_book = "/Volumes/workspace/dataanalysispysparkbook/bronze_files/"
path_pride_and_prejudice = "/Volumes/workspace/dataanalysispysparkbook/bronze_files/1342-0.txt"
path_silver = "/Volumes/workspace/dataanalysispysparkbook/silver/"

import pyspark.sql.functions as F

In [0]:
results = (
  spark.read.text(path_pride_and_prejudice)
            .select(F.split(F.col("value"), " ").alias("line"))
            .select(F.explode(F.col("line")).alias("word"))
            .select(F.lower(F.col("word")).alias("word"))
            .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
            .where(F.col("word") != "")
            .groupby("word")
            .count()
            .orderBy(F.col("count").desc())
)
results.show(5)

#### Exercise 3.3
1. By modifying the word_count_submit.py program, return the number of distinct
words in Jane Austen’s Pride and Prejudice. (Hint: results contains one
record for each unique word.).
2. (Challenge) Wrap your program in a function that takes a file name as a parameter.
It should return the number of distinct words.

In [0]:
distinct_words = results.select(F.countDistinct(F.col("word")).alias("distinct_words"))
distinct_words.show()

In [0]:
def distinct_words(file_name: str)-> int:
    results = (
        spark.read.text(f"{path_one_book}{file_name}")
                .select(F.split(F.col("value"), " ").alias("line"))
                .select(F.explode(F.col("line")).alias("word"))
                .select(F.lower(F.col("word")).alias("word"))
                .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
                .where(F.col("word") != "")
                .select(F.countDistinct(F.col("word")).alias("distinct_words"))
        )
    return results.first()

In [0]:
distinct_words("11-0.txt")

#### Exercise 3.4
Taking word_count_submit.py, modify the script to return a sample of five words that
appear only once in Jane Austen’s Pride and Prejudice.

In [0]:
five_words_appear_only_once = results.filter(F.col("count") == 1)
five_words_appear_only_once.show(5)

#### Exercise 3.5
1. Using the substring function (refer to PySpark’s API or the pyspark shell if needed), return the top five most popular first letters (keep only the first letter of each word).
2. Compute the number of words starting with a consonant or a vowel. (Hint: The isin() function might be useful.)

In [0]:
top_five_letters = (
                    results.select(F.col("word").substr(1, 1).alias("letter"), F.col("count"))
                            .groupby("letter")
                            .sum("count")
                            .orderBy(F.col("sum(count)").desc())
                )
top_five_letters.show(5)

In [0]:
count_vowel_consonants = (
    results.withColumn(
        "letter_type",
        F.when(F.col("word").isin(["a", "e", "i", "o", "u"]), "vowel").otherwise("consonant")
    )
    .groupBy("letter_type")
    #.sum("count")
    #.orderBy(F.col("sum(count)").desc())
    .agg(F.sum("count").alias("total_count"))
    .orderBy(F.col("total_count").desc())
)            
count_vowel_consonants.show()
